In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 16
tree_depth = 10
batch_size = 512
device = 'cpu'
train_data_path = r'/mnt/qnap/ekosman/mitbih_train.csv'
test_data_path = r'/mnt/qnap/ekosman/mitbih_test.csv'

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [ ]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

/home/eitan.k/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.475297927856445 | KNN Loss: 5.602973937988281 | CLS Loss: 1.8723238706588745
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.45629358291626 | KNN Loss: 3.5741217136383057 | CLS Loss: 0.8821719288825989
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.092979431152344 | KNN Loss: 3.3522326946258545 | CLS Loss: 0.7407468557357788
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 3.971721649169922 | KNN Loss: 3.3088138103485107 | CLS Loss: 0.6629079580307007
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 3.879157781600952 | KNN Loss: 3.311636447906494 | CLS Loss: 0.567521333694458
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 3.7825379371643066 | KNN Loss: 3.2636935710906982 | CLS Loss: 0.5188443660736084
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 3.724571704864502 | KNN Loss: 3.206364154815674 | CLS Loss: 0.5182076692581177
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 3.773526906967163 | KNN Loss: 3.2495923042297363 | CL

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.215547800064087 | KNN Loss: 3.0493133068084717 | CLS Loss: 0.16623446345329285
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.2456350326538086 | KNN Loss: 3.114419937133789 | CLS Loss: 0.13121512532234192
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.1848208904266357 | KNN Loss: 3.06929874420166 | CLS Loss: 0.11552203446626663
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.2359862327575684 | KNN Loss: 3.1000452041625977 | CLS Loss: 0.13594093918800354
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.1889703273773193 | KNN Loss: 3.0894834995269775 | CLS Loss: 0.09948676079511642
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.240623950958252 | KNN Loss: 3.073805093765259 | CLS Loss: 0.16681893169879913
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.223127603530884 | KNN Loss: 3.0912883281707764 | CLS Loss: 0.1318393498659134
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.1962461471557617 | KNN Loss: 3.101

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.1254847049713135 | KNN Loss: 3.018068313598633 | CLS Loss: 0.10741640627384186
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.1957969665527344 | KNN Loss: 3.050910472869873 | CLS Loss: 0.1448865830898285
Epoch: 007, Loss: 3.1540, Train: 0.9721, Valid: 0.9689, Best: 0.9712
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.1672019958496094 | KNN Loss: 3.0664424896240234 | CLS Loss: 0.1007595956325531
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.163222074508667 | KNN Loss: 3.0432236194610596 | CLS Loss: 0.11999840289354324
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.1639440059661865 | KNN Loss: 3.0616989135742188 | CLS Loss: 0.10224498063325882
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.184519052505493 | KNN Loss: 3.0765140056610107 | CLS Loss: 0.10800512880086899
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.1195497512817383 | KNN Loss: 3.0272727012634277 | CLS Loss: 0.09227711707353592
Epoch 8 / 200 | ite

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.091186046600342 | KNN Loss: 3.039804697036743 | CLS Loss: 0.05138140916824341
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.107962131500244 | KNN Loss: 3.018915891647339 | CLS Loss: 0.08904634416103363
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.0603342056274414 | KNN Loss: 3.002293586730957 | CLS Loss: 0.05804058536887169
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.1454226970672607 | KNN Loss: 3.0462677478790283 | CLS Loss: 0.09915491193532944
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.116126298904419 | KNN Loss: 3.0472683906555176 | CLS Loss: 0.06885800510644913
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.162749767303467 | KNN Loss: 3.047560691833496 | CLS Loss: 0.11518895626068115
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.1130826473236084 | KNN Loss: 2.9956142902374268 | CLS Loss: 0.11746841669082642
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.0878806114196777 | KNN Loss: 

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.0825815200805664 | KNN Loss: 3.002673387527466 | CLS Loss: 0.079908087849617
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.0879812240600586 | KNN Loss: 3.0278236865997314 | CLS Loss: 0.06015755236148834
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.106348991394043 | KNN Loss: 3.0556702613830566 | CLS Loss: 0.05067875236272812
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.0995595455169678 | KNN Loss: 3.0306074619293213 | CLS Loss: 0.06895215064287186
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.1364939212799072 | KNN Loss: 3.057478904724121 | CLS Loss: 0.07901498675346375
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.1579861640930176 | KNN Loss: 3.043156385421753 | CLS Loss: 0.1148298904299736
Epoch: 014, Loss: 3.0993, Train: 0.9814, Valid: 0.9778, Best: 0.9780
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.1010653972625732 | KNN Loss: 3.048903226852417 | CLS Loss: 0.05216224864125252
Epoch 15 /

Epoch 18 / 200 | iteration 10 / 171 | Total Loss: 3.1107540130615234 | KNN Loss: 3.0310940742492676 | CLS Loss: 0.07966001331806183
Epoch 18 / 200 | iteration 20 / 171 | Total Loss: 3.1205809116363525 | KNN Loss: 3.0421838760375977 | CLS Loss: 0.07839705049991608
Epoch 18 / 200 | iteration 30 / 171 | Total Loss: 3.087979316711426 | KNN Loss: 3.042618751525879 | CLS Loss: 0.04536059871315956
Epoch 18 / 200 | iteration 40 / 171 | Total Loss: 3.0854392051696777 | KNN Loss: 3.048264503479004 | CLS Loss: 0.03717474266886711
Epoch 18 / 200 | iteration 50 / 171 | Total Loss: 3.046215534210205 | KNN Loss: 3.00057315826416 | CLS Loss: 0.04564239829778671
Epoch 18 / 200 | iteration 60 / 171 | Total Loss: 3.084365129470825 | KNN Loss: 3.030977725982666 | CLS Loss: 0.05338743329048157
Epoch 18 / 200 | iteration 70 / 171 | Total Loss: 3.0990936756134033 | KNN Loss: 3.0539746284484863 | CLS Loss: 0.045119039714336395
Epoch 18 / 200 | iteration 80 / 171 | Total Loss: 3.0946459770202637 | KNN Loss: 2.

Epoch 21 / 200 | iteration 80 / 171 | Total Loss: 3.0480027198791504 | KNN Loss: 3.0246052742004395 | CLS Loss: 0.023397427052259445
Epoch 21 / 200 | iteration 90 / 171 | Total Loss: 3.0563135147094727 | KNN Loss: 3.0131585597991943 | CLS Loss: 0.043155066668987274
Epoch 21 / 200 | iteration 100 / 171 | Total Loss: 3.068023204803467 | KNN Loss: 3.0137362480163574 | CLS Loss: 0.05428692325949669
Epoch 21 / 200 | iteration 110 / 171 | Total Loss: 3.0750768184661865 | KNN Loss: 3.0397398471832275 | CLS Loss: 0.035336896777153015
Epoch 21 / 200 | iteration 120 / 171 | Total Loss: 3.083533525466919 | KNN Loss: 3.0370914936065674 | CLS Loss: 0.046442076563835144
Epoch 21 / 200 | iteration 130 / 171 | Total Loss: 3.0372166633605957 | KNN Loss: 2.9934444427490234 | CLS Loss: 0.04377210885286331
Epoch 21 / 200 | iteration 140 / 171 | Total Loss: 3.061859369277954 | KNN Loss: 3.006035327911377 | CLS Loss: 0.055824097245931625
Epoch 21 / 200 | iteration 150 / 171 | Total Loss: 3.085111618041992 |

In [ ]:
test(model, test_data_iter, device)

In [ ]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [ ]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

In [ ]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [ ]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

In [ ]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [ ]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [ ]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [ ]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [ ]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [ ]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [ ]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")